In [1]:
our_assembly = "/mnt/projects/pasteur_bacterias/refs/salmon1502.fasta"
ref_assembly = "/mnt/projects/pasteur_bacterias/refs/salmon_ref.fasta"

In [2]:
def sc_iter_fasta_for_refseq(file_name):
    """ Iter over fasta file."""

    header = None
    seq = []
    with open(file_name) as fh:
        data = fh.readlines()
        for line in data:
            if line.startswith(">"):
                if seq:
                    sequence = "".join(seq).upper()
                    yield (header, sequence)
                header = line.strip()[1:]
                seq = []
                continue
            seq.append(line.upper().strip())
        if seq or header:
            sequence = "".join(seq).upper()
            yield (header, sequence)
            
def get_revcomp(sequence):
    '''Return reverse complementary sequence.

    >>> complementary('AT CG')
    'CGAT'

    '''
    c = dict(zip('ATCGNatcgn~[]', 'TAGCNtagcn~]['))
    return ''.join(c.get(nucleotide, '') for nucleotide in reversed(sequence))

In [3]:
from collections import defaultdict

def get_position_graph(sequences, k=23):
    graph = defaultdict(list)
    for sid,sequence in enumerate(sequences):
        for i in range(len(sequence)-k+1):
            kmer = sequence[i:i+k]
            graph[kmer].append((sid,i))
            graph[get_revcomp(kmer)].append((sid,-1*i))
    return graph

In [4]:
our_assembly_sequences = []
our_assembly_headers = []
for header,seq in sc_iter_fasta_for_refseq(our_assembly):
    our_assembly_sequences.append(seq)
    our_assembly_headers.append(header)

ref_assembly_sequences = []
ref_assembly_headers = []
for header,seq in sc_iter_fasta_for_refseq(ref_assembly):
    ref_assembly_sequences.append(seq)
    ref_assembly_headers.append(header)

In [5]:
position_ours = get_position_graph(our_assembly_sequences)

In [6]:
position_ref = get_position_graph(ref_assembly_sequences)

In [7]:
print position_ours.keys()[:100]

SyntaxError: invalid syntax (<ipython-input-7-47df570dfa20>, line 1)

In [ ]:
position_ours["CATATTTACACACCTGCGCCAGC"], position_ref["CATATTTACACACCTGCGCCAGC"]

In [ ]:
common_kmers = set()
for kmer in position_ours:
    if position_ref[kmer]:
        common_kmers.add(kmer)

In [ ]:
len(common_kmers), len(position_ours), len([x for x in position_ref if position_ref[x]])

In [ ]:
k = 23
for sid, seq in enumerate(our_assembly_sequences):
    header = our_assembly_headers[sid]
    for i in range(len(seq)-k+1):
        kmer = seq[i:i+k]
        if i > 500:
            break
        if len(position_ours[kmer]) == 1 and len(position_ref[kmer]) == 1:
            print header, sid, i,kmer,position_ref[kmer]

In [ ]:
k = 23
last_sid = None
last_pos = 0


dataset = []
for sid, seq in enumerate(ref_assembly_sequences):
    header = ref_assembly_headers[sid]
    for i in range(len(seq)-k+1):
        kmer = seq[i:i+k]
        if len(position_ref[kmer]) == 1 and len(position_ours[kmer]) == 1:
            our_sid, our_pos = position_ours[kmer][0]
            if last_sid != our_sid:
                dataset.append([i, i-last_pos, kmer, our_sid, our_assembly_headers[our_sid], our_pos])
                last_sid = our_sid
                last_pos = i
dataset.append([i, i-last_pos, kmer, our_sid, our_assembly_headers[our_sid], our_pos])

In [ ]:
fixed_dataset = []
for i, x in enumerate(dataset):
    if i == len(dataset)-1:
        continue
    x[1] = dataset[i+1][1]

In [ ]:
for i, x in enumerate(dataset):
    if x[3] in [13]:
        continue
    if x[1] < 10000:
        continue
    print x

In [ ]:
our_assembly_headers[13],our_assembly_headers[11],our_assembly_headers[18]